In [ ]:
import os
from glob import glob

import cv2
import numpy as np
from sklearn.metrics import f1_score
from tqdm import tqdm

from metrics import compute_iou
from model import SegmentModel

In [ ]:
model_path: str = "./segment_model.h5"
data_path: str = "D:/DataSet/Figaro1K/"
device: str = "/gpu:0"
input_shape: tuple = (256, 256, 3)

In [ ]:
# loading the model instance
model = SegmentModel(
    model_path=model_path, 
    custom_objects={},
    input_shape=input_shape,
    device=device
)

In [ ]:
x_data_path: list = sorted(glob(os.path.join(data_path, "Original", "Testing", "*")))
y_data_path: list = sorted(glob(os.path.join(data_path, "GT", "Testing", "*")))
assert len(x_data_path) == len(y_data_path)

In [ ]:
n_samples: int = len(x_data_path)

x_data = np.zeros((n_samples,) + input_shape, dtype=np.uint8)
y_data = np.zeros((n_samples,) + input_shape[:2], dtype=np.uint8)

for i in tqdm(range(n_samples)):
    img = cv2.imread(x_data_path[i])[..., ::-1]
    x_data[i] = cv2.resize(img, input_shape[:2])
    
    label = cv2.imread(y_data_path[i])[..., ::-1]
    label = cv2.resize(label, input_shape[:2])
    label[label < 128] = 0
    label[label != 0] = 255
    y_data[i] = cv2.cvtColor(label, cv2.COLOR_RGB2GRAY)

In [ ]:
m_IoUs = []
f1_scores = []
for i in tqdm(range(n_samples)):
    hair_segment = model.get_mask(x_data[i], get_hair=True)
    hair_segment = cv2.cvtColor(hair_segment, cv2.COLOR_RGB2GRAY).squeeze()
    
    y_true = y_data[i] / 255
    y_pred = hair_segment / 255
    
    m_IoU = compute_iou(y_pred, y_true)
    f1 = f1_score(y_true=y_true, y_pred=y_pred, average='macro')
    
    m_IoUs.append(m_IoU)
    f1_scores.append(f1)

In [ ]:
final_f1_score = np.mean(np.asarray(f1_scores))
final_m_IoU =  np.mean(np.asarray(m_IoUs))

print("[*] F1 Score : {}".format(final_f1_score))
print("[*] mIoU     : {}".format(final_m_IoU))